In [1]:
# Import normal libraries
from datetime import datetime as dt
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras import metrics
import tensorflow as tf
# https://github.com/adriangb/scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor


# Import own libraries
from data_api.init_db import initialize_db
from data_api.init_db import return_engine
from data_api.db import tickers_to_ticker_ids, get_all_price_data_sets, load_formatted_train_data
from models.model_func import create_train_test_arrays

# Define the data paths
current_dir = os.getcwd()
database_dir = os.path.join(current_dir, "data")
db_filename = "database.db"
model_dir = os.path.join(current_dir, "data/models")
scaler_dir = os.path.join(current_dir, "data/scalers")

# Create the required folders, if they are not available
if not os.path.exists(database_dir):
    os.makedirs(database_dir)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(scaler_dir):
    os.makedirs(scaler_dir)
if os.path.exists(os.path.join(database_dir, db_filename)):
    # If the database already exits, get a connection to the database
    engine = return_engine(database_dir, db_filename=db_filename)
else:
    # Otherwise, create a new database and initialize it with data
    engine = initialize_db(database_dir,  db_filename=db_filename)

# Problem Introduction

From the Yahoo Finance API, we get the 6 different values for each day. High price, low price, open price, close price, adjusted close price and volume. The 5 price data variables are all closely linked together, and only the adjusted close price reflects additional events like divident payouts or stock splits. Therefore, we only consider the adjusted close price (now referred to as "price") and the volume for our models. 

Our goal is to predict future prices based on the past prices and the past volume. We can use the price/volume data of multiple stocks to predict the price/volume data of those multiple stocks to make use of the information hidden in the dependence of the development in those different assets. The problem is, that we have indeed a long timerange of past data, but we do not know anything about future prices (yet). Therefore, we need to split the past data into small sections and consider the earlier part of that section as the past data and the later part as the future. Therefore we can create many training sets of past/future data combinations to use for our training.

# Metrics

We need to define a useful metric or multiple metrics to evaluate the performance of our model and to compare different models with different hyperparameters. The goal of our model is to predict continious values, therefore we have a regression problem. For regression problems, we can use root mean squared error (RMSE) as a metric to optimize our model for. This metric weights larger deviations from the expected results more than lower deviations, but still has a value on the same scale as the output data.

- https://keras.io/api/metrics/
- https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/

# EDA (Exploratory data analysis)

When we check the data, we encounter one problem: We do not have price and volume data available for arbitrary dates for each asset. For weekends and holidays, there is usually no trading occuring on stock exchanges and therefore no price is determined. These dates without price data vary between different assets, depending on which exchanges they are listed and in which jurisdiction the exchanges are located. For some assets, like Bitcoin, there is nevertheless price data available for all days because there is less regulation about the exchanges and the cryptocurrencies can be easily traded completly digital. Additionally, there is no price data available before the asset was available. In the case of a publicly traded company, this is usually the IPO.
To train a model with different assets, we are therefore likely facing missing values for some assets on some dates. For the first, problem, namely missing days for holidays/weekends, we have two options:
- Drop the data for a specific date, when no price data is missing for at least one asset
- Fill the missing prices with the average price of the days before and after

In the first case, we would potentially reduce our dataset significant, in case we have many different assets from many differen exchanges/jurisdictions. Therefore we take the second option and fill up the missing prices as mentioned.

To handle the second problem, namely missing asset prices because the asset was not publicly traded, it is more difficult to handle this. We can either:
- Limit the oldest date to the date when, the last asset was available
- Assume an average price of the asset for the time before it was available
- Assume a price of 0 before the asset was available
- Assume the price of the first day when the asset was available



In [84]:
start_date=dt(2021,1,1)
end_date=dt.today()
tickers = ["GOOG"]
#tickers = ["AAPL", "BTC-USD"]
# tickers = ["GOOG", "AAPL", "BTC-USD"]
tickers_ids = tickers_to_ticker_ids(engine, tickers)
# Load the data
data = load_formatted_train_data(engine, tickers_ids, start_date, end_date, return_timestamp=False)

# Only for  Google Colab

In [73]:
# data.to_csv("data_big.csv", index = False)

In [74]:
# from google.colab import drive
# drive.mount('/content/drive')

In [75]:
# data_small_path = "/content/drive/MyDrive/Colab Notebooks/data_small.csv"
# data_tiny_path = "/content/drive/MyDrive/Colab Notebooks/data_very_small.csv"
# data_big_path = "/content/drive/MyDrive/Colab Notebooks/data_big.csv"
# data_tiny_path =  "data_very_small.csv"
# data_small_path = "data_small.csv"

In [76]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Dropout
# from keras import metrics
# import tensorflow as tf
# from sklearn.model_selection import GridSearchCV

# data = pd.read_csv(data_small_path)
# data.head()
# data.shape

# Modelling


To model our problem based on time series data, we need to recognize that the sequence of the data is also containing a lot of information and need to be used in the model. For this type of problem, a long short-term memory (LSTM) model is usually the best fit. With this model, the sequence data is processed in sequence and the data contained in the earlier sequence steps is kept during the whole process and has an influence on the prediction.

If we look at our training, data, we recognize different magnitudes especially when we compare the prices in dollars and the volume data in pieces. Additionally, the prices of stocks are also arbitrary, depending on how many single stock represent the ownership of the full company. Therefore we normalize the dataset to remove these huge ranges of values.

In [85]:
# Normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(data)
df_for_training_scaled = scaler.transform(data)

Additional layers -> decrease performance

In [86]:
def get_model(input_shape, output_shape, activation="relu", init_mode='uniform', learning_rate=0.001,beta_1=0.9,beta_2=0.999,amsgrad=False, dropout_rate=0.2):
    """
    Returns a predefined model object
    """
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, amsgrad=amsgrad)
    model = Sequential()
    model.add(LSTM(32, activation=activation, input_shape=(
        input_shape[1], input_shape[2]), return_sequences=True, kernel_initializer=init_mode))
    model.add(LSTM(32, activation=activation, return_sequences=False, kernel_initializer=init_mode))
    model.add(Dropout(dropout_rate))
    # If non-negative return values are required, this should be accomplished by the layers in the network. Anyway, with only non-negative input values, negative output values are very unlikely.
    # The relu activation function only returns positive values. -> This returns the same values for each predicted date
    model.add(Dense(output_shape[2]))  # ,  W_constraint=nonneg()))
    model.compile(optimizer=optimizer, loss='mean_squared_error',metrics=[
        metrics.RootMeanSquaredError(),
    ])
    return model

In [90]:
n_past = 120
trainX, trainY = create_train_test_arrays(n_past=n_past, df=df_for_training_scaled)
model = get_model(input_shape=trainX.shape, output_shape=trainY.shape, activation="relu", init_mode='uniform', learning_rate=0.01,beta_1=0.9, beta_2=0.89,amsgrad=True, dropout_rate=0.8)
history = model.fit(trainX, trainY, epochs=50, batch_size=128, validation_split=0.1, verbose=1)

Train dataset was successfully created:
trainX shape == (209, 120, 2)
trainY shape == (209, 1, 2)
Epoch 1/50
2/2 [==============================] - 2s 313ms/step - loss: 0.8262 - root_mean_squared_error: 0.9231 - val_loss: 0.3554 - val_root_mean_squared_error: 0.5961
Epoch 2/50
2/2 [==============================] - 0s 84ms/step - loss: 0.7858 - root_mean_squared_error: 0.8933 - val_loss: 0.3228 - val_root_mean_squared_error: 0.5681
Epoch 3/50
2/2 [==============================] - 0s 95ms/step - loss: 0.7134 - root_mean_squared_error: 0.8611 - val_loss: 0.3189 - val_root_mean_squared_error: 0.5647
Epoch 4/50
2/2 [==============================] - 0s 132ms/step - loss: 0.7131 - root_mean_squared_error: 0.8355 - val_loss: 0.3041 - val_root_mean_squared_error: 0.5514
Epoch 5/50
2/2 [==============================] - 0s 88ms/step - loss: 0.6993 - root_mean_squared_error: 0.8383 - val_loss: 0.2888 - val_root_mean_squared_error: 0.5374
Epoch 6/50
2/2 [==============================] - 0s 86

We still face issues with nan results in some cases. We need to tackle the problem of exploding gradiends more seriously.

Measure 1: Increase batch size:
- from 64 to 128: still nan
- from 64 to 256:still nan

- reduced learning rate to 0.01, batch_size to 128 and dropout to 0.8 -> Helped
- one layer less seems to have a quite good effect

We have very different model behaivior, depending if we use one stock, two or mulitle stocks and which timerange we use.

- Two LSTM layers with 32 nodes
- learning_rate of 0.01
- dropout of 0.8
- batch_size of 128
- epochs: 50

An important decision to make is how many past data sets (each data set representing one date) are used to predict the following data set. This influences how we create our training data set. We tested out different values and the result is, that a quite good solution is 120.

In [28]:
def compare_day_ranges_training(day_ranges_training, data)
    """
    Creates different data sets with different day ranges to train for the data of the next day.
    Trains a model for each day-range-dataset and returns a dictionary containing the history of the models training, indexed by the day_range
    """
    results = {}

    for day_range in day_ranges_training:
        print(f"Calculating for {day_range} ....")
        trainX, trainY = create_train_test_arrays(n_past=day_range, df=data)

        model = get_model(input_shape=trainX.shape, output_shape=trainY.shape)
        # model.summary()

        # fit the model
        history = model.fit(trainX, trainY, epochs=30, batch_size=64, validation_split=0.1, verbose=0)
        
        results[day_range] = history
        return results

In [ ]:
def evaluate_day_range_training(results, day_ranges_training):
    f = plt.figure()
    f.set_figwidth(15)
    f.set_figheight(10)
    for day_range in day_ranges_training:
        plt.plot(results[day_range].history['root_mean_squared_error'], label = str(day_range))
        plt.legend(loc="upper right")
        plt.title('model root_mean_squared_error')
        plt.ylabel('value')
        plt.xlabel('epoch')
    plt.show()

In [ ]:
day_ranges_training = [30, 60, 90, 120, 180, 200]
day_range_result = compare_day_ranges_training(day_ranges_training)

In [30]:
n_past = 120
trainX, trainY = create_train_test_arrays(n_past=n_past, df=df_for_training_scaled)

Train dataset was successfully created:
trainX shape == (462, 120, 2)
trainY shape == (462, 1, 2)


# Hyperparameter tuning

Now that we have our dataset ready, we need to find out what is a good model to predict the data set for the next price based on the last 120 data sets. After trying out several amounts of layers, we conclude that having two layers with 64 and 32 nodes leads to a quite good result. We choose the adam optimizer because it is known to be a good and efficient optimizer for regression problems.

We can use GridSearchCV to further optimize our model. First, we check different combinations of batch_sizes and epochs to see which combinations leads to the best result.

In [31]:
def evaluate_gridsearch(result):
    """
    Prints an formatted output for the GridSeachCV history result
    """
    print('Best Estimator: %s' % result.best_estimator_)
    print('Best Hyperparameters: %s' % result.best_params_)
    print(f'Best Score: {result.best_score_}') # score according to the metric we passed in refit

In [32]:
# Wrap the model to use the GridSearchCV library from sklearn
model = KerasRegressor(model=get_model, input_shape=trainX.shape, output_shape=trainY.shape, activation="relu", init_mode='uniform', optimizer = "adam", dropout_rate=0.2)

## Batch size and Epochs
We test different values for batch size and epoch using GridSearch to find the best combination

In [33]:
def run_grid_search(search_space, estimator, n_jobs = -1):
    """
    Runs GridSearch on the defined search_space. It starts with maximum amount of CPU cores and reduces the amount of used cores step by step if memory problems occur. Returns the history object from the GridSearchCV algorithm.
    """
    successfull = False
    while not successfull:
        #try:
        GS = GridSearchCV(estimator = model,
                    param_grid = search_space,
                    scoring = "neg_root_mean_squared_error", # With Sklearn, there is a convention that a higher score is always the better score, therefore root mean squared error is available as a negated score
                    cv = 5,
                    refit = True,
                    n_jobs = n_jobs, # Use all available threads
                #    error_score="raise",
                    verbose = 2)
        history = GS.fit(trainX, trainY[:,0,:])
        successfull = True
        print("GridSearch successfull with {n_jobs} parallel jobs")
        return history
        # except E as e:
        #     n_jobs -= 1
        #     print(e)



In [34]:
search_space_batch_epoch = {
    "batch_size" : [4, 8, 16, 32, 64], # smaller batch size require a longer training process, larger batch sizes require more powerfull hardware
    "epochs" : [10, 25, 50, 100, 200, 250]
}

# batch_epoch_history = run_grid_search(search_space=search_space_batch_epoch, estimator = model)

In [35]:
# evaluate_gridsearch(batch_epoch_history)

## Result from training with 120 days.
Best Estimator: KerasRegressor(
	model=<function get_model at 0x7f7a2ad078b0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	input_shape=(462, 120, 2)
	output_shape=(462, 1, 2)
	activation=relu
	init_mode=uniform
	dropout_rate=0.2
)
Best Hyperparameters: {'batch_size': 64, 'epochs': 10}
Best Score: -0.7624822584759838

Interestingly, if more days are used as input data, the batch size for training increases. On the other hand, the amount of epochs used to optimize the model is reduced tremendously. Only 10 epochs lead to the optimal solutiion (within our search space) compared to 250 epochs with an expected further slight improvement with additional epochs.

This reduction in epochs can be explained due to the fact that a higher number of epochs lead to the exploding gradient decent problem. This is a issue that is typical for regression models, because the output is not bounded and the combination of big weights and multiple layers can lead to very high gradients in the first layer, which lead to a nan value in out training metric.

After running the GridSearchCV algorithm, we come to the best parameter combination:
- batch_size: 32
- epochs: 250

After running different combinations, we also realized that increasing the amount of epochs may improve the model a bit more, but it increases computation time extensively.
The best score for the neg_root_mean_squared_error is -0.6563984733875825, which leads to a root_mean_squared_error of 0.6563984733875825.

## Hyperparameters for adam optimizer
The adam optimizer has several different parameters which we can test and combine to find a more optimal solution for our problem than with the default adam parameters.

In [39]:
big_search_space = {
    "batch_size" : [4, 8, 16, 32, 64], # smaller batch size require a longer training process, larger batch sizes require more powerfull hardware
    "epochs" : [10, 25, 50, 100, 200, 250],
    "optimizer__learning_rate" : [0.001,  0.0001, 0.01, 0.1],
    "optimizer__beta_1" : [0.8, 0.9, 1.0],
    "optimizer__beta_2" : [0.89, 0.99, 1.0],
    "optimizer__amsgrad" : [True, False],
}
history_big_search_space = run_grid_search(big_search_space, estimator = model)

Fitting 5 folds for each of 2160 candidates, totalling 10800 fits


2022-04-23 12:57:20.310467: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 12:57:20.314230: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 12:57:20.422328: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
93/93 [==============================] - 20s 178ms/step - loss: nan - root_mean_squared_error: nanrror: 8705578.000
Epoch 2/10
93/93 [==============================] - 20s 174ms/step - loss: 4.1544 - root_mean_squared_error: 2.038271
Epoch 2/10
93/93 [==============================] - 22s 200ms/step - loss: nan - root_mean_squared_error: nan3815.24713332.5
Epoch 2/10
93/93 [==============================] - 22s 199ms/step - loss: nan - root_mean_squared_error: nan3793.75168573.00
Epoch 2/10
93/93 [==============================] - 22s 191ms/step - loss: nan - root_mean_squared_error: nan
Epoch 2/10
93/93 [==============================] - 22s 199ms/step - loss: 0.7232 - root_mean_squared_error: 0.8504
Epoch 2/10
93/93 [==============================] - 23s 203ms/step - loss: 66544755277824.0000 - root_mean_squared_error: 8157497.0000
Epoch 2/10
93/93 [==============================] - 23s 208ms/step

/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_r

[CV] END batch_size=4, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, optimizer__beta_2=0.89, optimizer__learning_rate=0.001; total time= 3.3min
24/24 [==============================] - 2s 40ms/stepss: 22934876.0000 - root_mean_squared_error: 4789.03
[CV] END batch_size=4, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, optimizer__beta_2=0.89, optimizer__learning_rate=0.001; total time= 3.4min
23/23 [==============================] - 1s 33ms/step
[CV] END batch_size=4, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, optimizer__beta_2=0.89, optimizer__learning_rate=0.0001; total time= 3.4min


...

/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_r

93/93 [==============================] - 24s 254ms/step - loss: nan - root_mean_squared_error: nan
Epoch 15/25
93/93 [==============================] - 24s 256ms/step - loss: nan - root_mean_squared_error: nan
Epoch 19/25
93/93 [==============================] - 26s 280ms/step - loss: nan - root_mean_squared_error: nan
Epoch 13/25
93/93 [==============================] - 25s 267ms/step - loss: nan - root_mean_squared_error: nan
Epoch 12/25
93/93 [==============================] - 25s 271ms/step - loss: 0.4463 - root_mean_squared_error: 0.668194992.125
Epoch 13/25
93/93 [==============================] - 25s 265ms/step - loss: nan - root_mean_squared_error: nanror: 673602.625
Epoch 15/25
93/93 [==============================] - 27s 287ms/step - loss: nan - root_mean_squared_error: nanror: 663622.7
Epoch 11/25
93/93 [==============================] - 26s 280ms/step - loss: nan - root_mean_squared_error: nanor: 480565.0
Epoch 16/25
93/93 [==============================] - 25s 272ms/step -

/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_r

93/93 [==============================] - 26s 279ms/step - loss: 0.6761 - root_mean_squared_error: 0.8223
Epoch 8/25
93/93 [==============================] - 25s 268ms/step - loss: nan - root_mean_squared_error: nan
Epoch 22/25
93/93 [==============================] - 25s 271ms/step - loss: nan - root_mean_squared_error: nan
Epoch 20/25
93/93 [==============================] - 24s 254ms/step - loss: 0.4398 - root_mean_squared_error: 0.6632
Epoch 20/25
93/93 [==============================] - 26s 281ms/step - loss: nan - root_mean_squared_error: nan
Epoch 19/25
93/93 [==============================] - 24s 253ms/step - loss: nan - root_mean_squared_error: nan                          
Epoch 22/25
93/93 [==============================] - 26s 276ms/step - loss: nan - root_mean_squared_error: nan
Epoch 18/25
93/93 [==============================] - 29s 262ms/step - loss: nan - root_mean_squared_error: nan
Epoch 2/25
93/93 [==============================] - 24s 259ms/step - loss: nan - root_m

KeyboardInterrupt: 

In [36]:
adam_parameters = {
    "optimizer__learning_rate" : [0.001,  0.0001, 0.01, 0.1],
    "optimizer__beta_1" : [0.8, 0.9, 1.0],
    "optimizer__beta_2" : [0.89, 0.99, 1.0],
    "optimizer__amsgrad" : [True, False],
    # already optimized parameters, which we like to use again for out training
    "batch_size" : [64],
    "epochs" : [10]
}
history_adam = run_grid_search(adam_parameters, estimator = model)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


2022-04-23 12:25:23.975090: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 12:25:24.063728: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 12:25:24.152712: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
6/6 [==============================] - 6s 260ms/step - loss: 0.7647 - root_mean_squared_error: 0.8745
Epoch 2/10
6/6 [==============================] - 5s 259ms/step - loss: 0.7644 - root_mean_squared_error: 0.8743
Epoch 2/10
6/6 [==============================] - 6s 281ms/step - loss: 0.8741 - root_mean_squared_error: 0.9349
Epoch 2/10
6/6 [==============================] - 6s 337ms/step - loss: 0.8746 - root_mean_squared_error: 0.9352
Epoch 2/10
6/6 [==============================] - 6s 283ms/step - loss: 0.7575 - root_mean_squared_error: 0.8704
Epoch 2/10
6/6 [==============================] - 6s 289ms/step - loss: 0.8695 - root_mean_squared_error: 0.9325
Epoch 2/10
6/6 [==============================] - 7s 426ms/step - loss: 0.6941 - root_mean_squared_error: 0.8331
Epoch 2/10
6/6 [==============================] - 2s 296ms/step - loss: 0.7154 - root_mean_squared_error: 0.8458
Epoch 3/10
6/6 [===

/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_r

6/6 [==============================] - 3s 509ms/step - loss: 0.4033 - root_mean_squared_error: 0.6351
Epoch 1/10
2/2 [==============================] - 1s 62ms/stepss: nan - root_mean_squared_error: nan0.707
[CV] END batch_size=64, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, optimizer__beta_2=0.89, optimizer__learning_rate=0.0001; total time=  36.3s
2/2 [==============================] - 1s 84ms/stepss: 0.4800 - root_mean_squared_error: 0.6928
[CV] END batch_size=64, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, optimizer__beta_2=0.89, optimizer__learning_rate=0.01; total time=  35.5s
2/2 [==============================] - 1s 91ms/stepss: nan - root_mean_squared_error: n
[CV] END batch_size=64, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, optimizer__beta_2=0.89, optimizer__learning_rate=0.1; total time=  34.6s
2/2 [==============================] - 1s 87ms/step
[CV] END batch_size=64, epochs=10, optimizer__amsgrad=True, optimizer__beta_1=0.8, o

...

2/2 [==============================] - 1s 40ms/stepss: 2.4637 - root_mean_squared_error: 1.5667
[CV] END batch_size=64, epochs=10, optimizer__amsgrad=False, optimizer__beta_1=1.0, optimizer__beta_2=1.0, optimizer__learning_rate=0.01; total time=  26.6s
6/6 [==============================] - 2s 281ms/step - loss: 2.2078 - root_mean_squared_error: 1.4859
Epoch 6/10
6/6 [==============================] - 2s 308ms/step - loss: 0.4679 - root_mean_squared_error: 0.6841
Epoch 6/10
6/6 [==============================] - 2s 311ms/step - loss: 0.8069 - root_mean_squared_error: 0.8983
Epoch 4/10
6/6 [==============================] - 2s 320ms/step - loss: 0.6078 - root_mean_squared_error: 0.7796
Epoch 8/10
6/6 [==============================] - 2s 266ms/step - loss: 13551.6113 - root_mean_squared_error: 116.4114
Epoch 7/10
6/6 [==============================] - 2s 285ms/step - loss: 391191428115464192.0000 - root_mean_squared_error: 625452928.0000
Epoch 5/10
6/6 [==============================] -

/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_r

6/6 [==============================] - 1s 123ms/step - loss: 0.4658 - root_mean_squared_error: 0.6825
Epoch 6/10
6/6 [==============================] - 1s 115ms/step - loss: 0.4956 - root_mean_squared_error: 0.7040
Epoch 6/10
6/6 [==============================] - 1s 112ms/step - loss: 0.4842 - root_mean_squared_error: 0.6958
Epoch 7/10
6/6 [==============================] - 1s 115ms/step - loss: 0.5108 - root_mean_squared_error: 0.7147
Epoch 7/10
6/6 [==============================] - 1s 114ms/step - loss: 0.4898 - root_mean_squared_error: 0.6998
Epoch 8/10
6/6 [==============================] - 1s 110ms/step - loss: 0.5225 - root_mean_squared_error: 0.7229
Epoch 8/10
6/6 [==============================] - 1s 111ms/step - loss: 0.4924 - root_mean_squared_error: 0.7017
Epoch 9/10
6/6 [==============================] - 1s 111ms/step - loss: 0.5200 - root_mean_squared_error: 0.7211
Epoch 9/10
6/6 [==============================] - 1s 127ms/step - loss: 0.4897 - root_mean_squared_error: 0

/home/martin/.conda/envs/stock_price_pred/lib/python3.9/site-packages/sklearn/model_selection/_search.py:969: UserWarning: One or more of the test scores are non-finite: [-6.78116458e-01             nan             nan -7.58690765e-01
             nan             nan             nan -6.95510374e-01
             nan -6.01614141e-01             nan             nan
             nan             nan             nan -5.84481605e-01
 -5.96009386e+03             nan             nan -6.88266081e-01
             nan             nan -7.05330880e-01 -7.87317917e-01
             nan             nan             nan -1.08643160e+02
             nan             nan             nan             nan
             nan             nan             nan             nan
 -7.25149756e-01             nan             nan             nan
             nan -6.65960665e-01             nan             nan
             nan             nan             nan             nan
             nan -7.14668031e-01             nan  

8/8 [==============================] - 2s 68ms/step - loss: 0.7788 - root_mean_squared_error: 0.8825
Epoch 2/10
8/8 [==============================] - 1s 68ms/step - loss: 0.6769 - root_mean_squared_error: 0.8227
Epoch 3/10
8/8 [==============================] - 1s 73ms/step - loss: nan - root_mean_squared_error: nan                   
Epoch 4/10
8/8 [==============================] - 1s 68ms/step - loss: nan - root_mean_squared_error: nan
Epoch 5/10
8/8 [==============================] - 1s 70ms/step - loss: nan - root_mean_squared_error: nan
Epoch 6/10
8/8 [==============================] - 1s 69ms/step - loss: nan - root_mean_squared_error: nan
Epoch 7/10
8/8 [==============================] - 1s 69ms/step - loss: nan - root_mean_squared_error: nan
Epoch 8/10
8/8 [==============================] - 1s 66ms/step - loss: nan - root_mean_squared_error: nan
Epoch 9/10
8/8 [==============================] - 1s 69ms/step - loss: nan - root_mean_squared_error: nan
Epoch 10/10
8/8 [=========

In [37]:
evaluate_gridsearch(history_adam)

Best Estimator: KerasRegressor(
	model=<function get_model at 0x7fdfbbea39d0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	input_shape=(462, 120, 2)
	output_shape=(462, 1, 2)
	activation=relu
	init_mode=uniform
	dropout_rate=0.2
	optimizer__amsgrad=True
	optimizer__beta_1=0.9
	optimizer__beta_2=0.89
	optimizer__learning_rate=0.1
)
Best Hyperparameters: {'batch_size': 64, 'epochs': 10, 'optimizer__amsgrad': True, 'optimizer__beta_1': 0.9, 'optimizer__beta_2': 0.89, 'optimizer__learning_rate': 0.1}
Best Score: -0.5844816053348458


We come to the conclusion, that our best parameters for 30 days are:
- optimizer__amsgrad: False
- optimizer__beta_1: 0.8
- optimizer__beta_2: 0.89
- optimizer__learning_rate: 0.01

The best score for the neg_root_mean_squared_error is -0.8073114933087524, which leads to a root_mean_squared_error of 0.8073114933087524.

Best Estimator: KerasRegressor(
	model=<function get_model at 0x7fdfbbea39d0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	input_shape=(462, 120, 2)
	output_shape=(462, 1, 2)
	activation=relu
	init_mode=uniform
	dropout_rate=0.2
	optimizer__amsgrad=True
	optimizer__beta_1=0.9
	optimizer__beta_2=0.89
	optimizer__learning_rate=0.1
)
Best Hyperparameters: {'batch_size': 64, 'epochs': 10, 'optimizer__amsgrad': True, 'optimizer__beta_1': 0.9, 'optimizer__beta_2': 0.89, 'optimizer__learning_rate': 0.1}
Best Score: -0.5844816053348458


We come to the conclusion, that our best parameters for 120 days are:
- optimizer__amsgrad: True
- optimizer__beta_1: 0.9
- optimizer__beta_2: 0.89
- optimizer__learning_rate: 0.1

The best score for the neg_root_mean_squared_error is -0.8073114933087524, which leads to a root_mean_squared_error of 0.8073114933087524.

# Results
After testing different LSTM networks, hyperparameters and training parameters either manually, or by the use of GridSearch, we can come to a conclusion for a quite good model:

- 120 days as input data to predict for day 121
- 2 LSTM layers with 64 and 32 nodes respectively
- Batch size for training of 32 and 250 training epochs. More epochs improve our result, but they lead to a lot of computation time. Use less epochs may be also sufficient.

# Improvements
There are possibilties to spend time and computation capacity to further improve the model. Furthermore, there are many more different optimizers as well as activation functions available, which have each individually countless different parameter combinations. These could be evaluated to see if there is a possibility to still improve the model.
Also, other techniques except GridSearch could be used to test different sets of parameters. There is a library called RandomSearch which is similar to GridSearch, but creates more random combination by using ranges of values for the parameters.
A different option woult be the usage of Genetic algorithms. These algorithms apply a model similar to evolution to the models to find a optimal set of parameters.

If the problem of exploding gradiends would be solved, then a training with additional epochs could be used to further improve the models. Details to that problem can be found here: https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network

# Conclusion/Reflection
Training machine learning models is a very time intensive process, if it is done wrong. Therefore I come to the following conclusions to speed up development in the future:
- Test all scripts etc. with small amounts of data to verify that everything runs correct to avoid breakages after hours of model training and grid search
- Catch errors and so that you code can finish to run without breaking
- Use Multithreading if possible
- Run scripts overnight to not waste time waiting for the result
- Run scripts on a server so that it can run without occuping local resources and wearing down a laptops CPU